In [1]:
import pysam
from itertools import islice

# Checking how the coverage is calculated

Attempt to determine details of the coverage calculation. We use the Picard tool CollectWgsMetrics to calculate the coverage.

The experiment consists of two components: `script.sh` and this notebook.

**The experiment was a failure, and an alternative method -- reading the source code -- was used to arrive at a conclusion.**

Method:
* Extract a few reads from the FASTQ files and align them to the reference, to have a small SAM/BAM file to work with.
* Compute the genomic coverage (for information)
* Mark duplicates in the SAM file.
* Compute the coverage again as a baseline.
* Remove some records from the SAM file. The same set of records is analysed in this notebook.
* Compute the coverage of the new SAM file. Compare the difference with the metrics in this file.


# From the Picard documentation

https://gatk.broadinstitute.org/hc/en-us/articles/360036452852-CollectWgsMetrics-Picard-

The following options (defaults listed here) affect the computed coverage:
* `COUNT_UNPAIRED` False
* `MINIMUM_BASE_QUALITY`: 20
* `MINIMUM_MAPPING_QUALITY`: 20

Other options aren't relevant for the calculation of overlap.

In [2]:
f = pysam.AlignmentFile("test/sample.markdup.bam", "rb")

In [3]:
# Coverage calculation requires a big RAM to hold coverage for each position
#f.count_coverage("chr1")

# First read in the sorted file


In [11]:
reads = []
want = 8
for count, r in enumerate(f.fetch()):
    if r.mapping_quality < 20 or not r.is_proper_pair:
        print(r.query_name, "..skipped")
        continue
    else:
        print("\n", r.query_name)
    print(">", r.reference_name + ":" + str(r.pos))
    print("> R1:", r.is_read1, ", is_reverse:", r.is_reverse, ", is_proper_pair:", r.is_proper_pair,
         ", is_secondary:", r.is_secondary, ", is_duplicate:", r.is_duplicate)
    print("> mapping_quality:", r.mapping_quality)
    
    reads.append(r)
    if len(reads) == want:
        break

print("\n\nRead", count, "records from the file. Found", len(reads), "alignments satisfying the criteria.")

E00426:63:HYK7TCCXY:5:1104:24850:7304 ..skipped
E00426:63:HYK7TCCXY:5:1104:24850:7304 ..skipped
E00426:63:HYK7TCCXY:5:1104:20456:7462 ..skipped
E00426:63:HYK7TCCXY:5:1104:20456:7462 ..skipped

 E00426:63:HYK7TCCXY:5:1104:24495:7708
> chr1:623652
> R1: False , is_reverse: False , is_proper_pair: True , is_secondary: False , is_duplicate: False
> mapping_quality: 24

 E00426:63:HYK7TCCXY:5:1104:24495:7708
> chr1:624199
> R1: True , is_reverse: True , is_proper_pair: True , is_secondary: False , is_duplicate: False
> mapping_quality: 24
E00426:63:HYK7TCCXY:5:1104:20791:7058 ..skipped
E00426:63:HYK7TCCXY:5:1104:30076:7603 ..skipped
E00426:63:HYK7TCCXY:5:1104:30076:7603 ..skipped

 E00426:63:HYK7TCCXY:5:1104:17797:7110
> chr1:738788
> R1: False , is_reverse: False , is_proper_pair: True , is_secondary: False , is_duplicate: False
> mapping_quality: 44

 E00426:63:HYK7TCCXY:5:1104:17797:7110
> chr1:738948
> R1: True , is_reverse: True , is_proper_pair: True , is_secondary: False , is_duplica

Alignments are selected for further analysis in the next step.

# Analyse read pairs

Compute the overlap, etc.

In [5]:
read_pairs = zip(reads[::2], reads[1::2])
reads_bases = 0
reads_bases_no_overlap = 0
reads_q20_bases = 0

for i,(ra, rb) in enumerate(read_pairs):
    print("- pair", i+1, "-")
    insert_size = (rb.pos + rb.reference_length) - (ra.pos)
    print("Insert size:", insert_size)
    read_bases = ra.reference_length + rb.reference_length
    print("Combined aligned ref length:", read_bases, "(", ra.reference_length, "+", rb.reference_length,")")
    print("Overlap:", read_bases - insert_size)
    reads_bases += read_bases
    reads_bases_no_overlap += min(insert_size, read_bases)
    q20_bases_a = sum(q >= 20 for q in ra.query_alignment_qualities)
    q20_bases_b = sum(q >= 20 for q in rb.query_alignment_qualities)
    print("Covered at Q>=20:", q20_bases_a + q20_bases_b, "(", q20_bases_a, "+", q20_bases_b, ")")
    reads_q20_bases += q20_bases_a + q20_bases_b
    print("")


print("")
print("total bases:           ", reads_bases)
print("uniquely covered bases:", reads_bases_no_overlap)
print("bases at Q >= 20:      ", reads_q20_bases)

- pair 1 -
Insert size: 698
Combined aligned ref length: 302 ( 151 + 151 )
Overlap: -396
Covered at Q>=20: 255 ( 108 + 147 )

- pair 2 -
Insert size: 311
Combined aligned ref length: 302 ( 151 + 151 )
Overlap: -9
Covered at Q>=20: 283 ( 139 + 144 )

- pair 3 -
Insert size: 92
Combined aligned ref length: 135 ( 64 + 71 )
Overlap: 43
Covered at Q>=20: 123 ( 53 + 70 )

- pair 4 -
Insert size: 155
Combined aligned ref length: 302 ( 151 + 151 )
Overlap: 147
Covered at Q>=20: 243 ( 120 + 123 )


total bases:            1041
uniquely covered bases: 851
bases at Q >= 20:       904


## Analysis of the last two read pairs

Both reads in pair 3 are soft-clipped in both ends (before and after match to reference)! The alignment is probably spurious / bogus. The actual insert size reported in IGV is 43, taking into account the soft clipping. 

The number of bases with Q=>20 -- 123 -- is too high to allow the possibility that doubly covered bases are only counted once / given a quality score of 0:

In [6]:
overlap = 43
ra_once_covered = 64 - overlap
rb_once_covered = 71 - overlap
print("Uniquely covered bases:", ra_once_covered + rb_once_covered + overlap)

Uniquely covered bases: 92


The fourth pair has a simpler configuration without soft-clipping. Almost all of the bases are overlapping because the insert size is only 155. The base quality >=20 is clearly not compatible with having overlapping bases set to zeros. The max possible should have been:

In [7]:
ra_ref_length, rb_ref_length = 151, 151
overlap = 147
print("expected maximum if only count covered bases once:",ra_ref_length + rb_ref_length - overlap)

expected maximum if only count covered bases once: 155


That is, the same as the insert size.

## Computation of the coverage using Picard


In [12]:
def get_picard_coverage(path):
    with open(path) as f:
        flag = False
        for l in f:
            parts = l.split('\t')
            if flag:
                return [float(p) for p in parts[0:2]]
            if parts[0:2] == ['GENOME_TERRITORY','MEAN_COVERAGE']:
                flag = True

In [13]:
territory, coverage_0 = get_picard_coverage('test/sample_wgs_metrics.txt')
print("Coverage before mark duplicates:", coverage_0)
_, coverage_full = get_picard_coverage('test/sample_wgs_metrics_post_markdup.txt')
print("Coverage after mark duplicates:", coverage_full)
_, coverage_tail = get_picard_coverage('test/tail_wgs_metrics.txt')
print("Coverage after removing the reads:", coverage_tail)

Coverage before mark duplicates: 0.001443
Coverage after mark duplicates: 0.001384
Coverage after removing the reads: 0.001384


# Conclusion

The output from Picard is not precise enough to make this analysis easy. The coverage has the "genome territory" -- the size of the genome -- in the denominator, and thus the difference in coverage caused by a few reads is a very small number.

Excluding more reads in the second run would be an option, but the next paragraph shows that it's not really necessary.


## Picard source code gives the answer

The CollectWgsMetrics source has a function `addInfo` that is called at every locus in the reference. It includes an if statement that ensures that only one read of each query name is counted. Read pairs have the same query name, and all other reads have different query names, so this ensures that overlap regions are only counted once.

https://github.com/broadinstitute/picard/blob/master/src/main/java/picard/analysis/CollectWgsMetrics.java#L425

The tool provides an output column `PCT_EXC_OVERLAP` that informs us of the fraction of bases excluded due to overlap.

## Summary of conditions for counting coverage

The following bases are excluded when computing the coverage:

* Adapter sequences
* All bases in reads with mapping quality < 20 (including unmapped reads)
* Duplicates
* Unpaired reads
* Secondary alignments
* With base quality < 20
* Overlap

There are also caps for max coverage to count at each given locus.

